In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch.nn as nn
import torch
import torchvision
from torchvision import models, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

import numpy as np
import random

%matplotlib inline
import matplotlib.pyplot as plt

from copy import deepcopy

basepath = '/content/drive/My Drive/CS618_Project/MNIST_LeNet300100'

In [0]:
class EarlyStopping:
    def __init__(self, patience=7, filename='checkpoint_0', verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.filename = filename

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose: print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(),basepath+'/CheckPoints/'+self.filename+'.pt')
        self.val_loss_min = val_loss

In [0]:
def get_data_loaders():
  transform = transforms.Compose([transforms.Resize((28, 28)),transforms.ToTensor(),])
  mnist_train = torchvision.datasets.MNIST(basepath,transform=transform,train=True,download=True)
  mnist_test = torchvision.datasets.MNIST(basepath,transform=transform,train=False,download=True)
  data_loader_train = torch.utils.data.DataLoader(mnist_train, batch_size=1024, shuffle=True, num_workers=8)
  data_loader_test = torch.utils.data.DataLoader(mnist_test, batch_size=4096, shuffle=True, num_workers=8)
  return(data_loader_train,data_loader_test)

data_loader_train,data_loader_test = get_data_loaders()

In [0]:
def defineMasks():
  mask_linear1 = torch.ones(300,784)
  mask_linear2 = torch.ones(100,300)
  mask_linear3 = torch.ones(10,100)
  return mask_linear1, mask_linear2, mask_linear3

In [0]:

class LeNet300100(nn.Module):
    def __init__(self):
        super(LeNet300100, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(784, 300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
            nn.LogSoftmax(dim=-1)
        )
    def forward(self, img):
        output = img.view(img.size(0), -1)
        output = self.classifier(output)
        return output


In [0]:
num_classes = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

mask_linear1, mask_linear2, mask_linear3 = defineMasks()

def get_unpruned_model(num_classes):
  model = LeNet300100()
  return(model)

model = get_unpruned_model(num_classes)

In [0]:
def plot_losses(train_loss,test_loss,filename,i):
  x_axis = list(range(1,len(train_loss)+1))
  fig = plt.figure(i,figsize=(20,10))
  plt.axes(yscale='log')
  plt.plot(x_axis, train_loss, color='blue', label='Train Loss Loss')
  plt.plot(x_axis, test_loss, color='green', label='Validation Loss')
  plt.legend(loc=1)
  plt.xlabel('Iterations over entire dataset')
  plt.ylabel('Loss')
  plt.ylim(0.01,3)
  plt.savefig(basepath+'/Losses/'+filename+'.png')
  plt.close(fig)

def plot_accuracies(train_accuracy,test_accuracy,filename,i):
  x_axis = list(range(1,len(train_accuracy)+1))
  fig = plt.figure(i,figsize=(20,10))
  plt.yscale('log')
  plt.plot(x_axis, train_accuracy, color='blue', label='Train Accuracy')
  plt.plot(x_axis, test_accuracy, color='green', label='Validation Accuracy')
  plt.legend(loc=1)
  plt.xlabel('Iterations over entire dataset')
  plt.ylabel('Accuracy')
  plt.ylim(0.7,1)
  plt.savefig(basepath+'/Accuracies/'+filename+'.png')
  plt.close(fig)

In [0]:
def train(model,prune_itr,lr_multiplier=1):
  
  scale_factor = 16

  learning_rate = 1e-3*lr_multiplier; wt_dcy =1e-7; lr_patience = int(500/scale_factor); lr_stepsize = int(1500/scale_factor); es_patience = int(4000/scale_factor)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=wt_dcy)
  scheduler_plateau = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=lr_patience, verbose=True, threshold=1e-7)
  scheduler_stepLR = StepLR(optimizer, step_size=lr_stepsize, gamma=0.8)
  early_stopping = EarlyStopping(patience=es_patience,filename='checkpoint_'+str(prune_itr), verbose=True)

  torch.cuda.empty_cache()
  model = model.to(device)
  num_epochs = 30
  es_flag = 0

  train_loss = []; test_loss = []; train_accuracy = []; test_accuracy = []
  
  best_loss = 100000; best_accuracy = 2

  for epoch in range(num_epochs):
    
    correct = 0; data_size = 0; t_loss = 0
    
    for i, (images, labels) in enumerate(data_loader_train):

      model.train()
      images = images.to(device); labels = labels.to(device)
      data_size += len(images)
      outputs = torch.nn.functional.log_softmax(model(images), dim=1)
      trainloss = criterion(outputs,labels)
      optimizer.zero_grad()
      trainloss.backward()
      optimizer.step()
      _, predicted = torch.max(outputs.data, 1)
      correct += (predicted == labels).sum().item()
      t_loss += trainloss.item()
    
      correct_t = 0; data_size_t = 0; tt_loss = 0
      model.eval()
      with torch.no_grad():
        for images, labels in data_loader_test:
          images = images.to(device); labels = labels.to(device)
          data_size_t += len(images)
          outputs = torch.nn.functional.log_softmax(model(images), dim=1)
          _, predicted = torch.max(outputs.data, 1)
          correct_t += (predicted == labels).sum().item()
          tt_loss += criterion(outputs, labels).item()
      print('Epoch: {}, Batch: {}, Train Loss: {:.3f} Train Accuracy: {:.3f}%, Validation Loss: {:.3f} Validation Accuracy: {:.3f}%'.
            format(epoch+1, i+1, trainloss.item(), 100*(predicted==labels).sum().item()/len(images), tt_loss/len(data_loader_test), 100*correct_t/data_size_t))
      
      train_loss.append(trainloss.item()); test_loss.append(tt_loss/len(data_loader_test))
      train_accuracy.append((predicted==labels).sum().item()/len(images)); test_accuracy.append(correct_t/data_size_t)
      
      if (tt_loss/len(data_loader_test)) < best_loss:
        best_loss = tt_loss/len(data_loader_test)
        best_accuracy = correct_t/data_size_t
        

      scheduler_stepLR.step()
      scheduler_plateau.step(tt_loss/len(data_loader_test))

      early_stopping(tt_loss/len(data_loader_test), model)
      if early_stopping.early_stop:
        print("Early stopping")
        es_flag = 1
        break
    if es_flag == 1: break
  
  model.load_state_dict(torch.load(basepath+'/CheckPoints/'+'checkpoint_'+str(prune_itr)+'.pt'))

  return(model,train_loss,test_loss,train_accuracy,test_accuracy,best_loss,best_accuracy)

In [10]:
lr_multiplier = 1
dict_best_info = {}
#model.load_state_dict(torch.load(basepath+'/CheckPoints/'+'checkpoint_0.pt'))
model,train_loss,test_loss,train_accuracy,test_accuracy,best_loss,best_accuracy = train(model,0,lr_multiplier)
dict_best_info[0] = (best_loss,best_accuracy)
plot_losses(train_loss,test_loss,'l0',1)
plot_accuracies(train_accuracy,test_accuracy,'a0',2)

model.load_state_dict(torch.load(basepath+'/CheckPoints/'+'checkpoint_0.pt'))
model = model.to(device)
print(model)

print('Best Accuracy is ' + str(round(best_accuracy*100,2)))
print('Best Loss is ' + str(round(best_loss,3)))

Epoch: 1, Batch: 1, Train Loss: 2.303 Train Accuracy: 22.069%, Validation Loss: 2.268 Validation Accuracy: 23.250%
Validation loss decreased (inf --> 2.267541).  Saving model ...
Epoch: 1, Batch: 2, Train Loss: 2.269 Train Accuracy: 31.914%, Validation Loss: 2.228 Validation Accuracy: 33.060%
Validation loss decreased (2.267541 --> 2.227639).  Saving model ...
Epoch: 1, Batch: 3, Train Loss: 2.227 Train Accuracy: 47.511%, Validation Loss: 2.180 Validation Accuracy: 47.230%
Validation loss decreased (2.227639 --> 2.180015).  Saving model ...
Epoch: 1, Batch: 4, Train Loss: 2.183 Train Accuracy: 60.675%, Validation Loss: 2.124 Validation Accuracy: 59.870%
Validation loss decreased (2.180015 --> 2.123863).  Saving model ...
Epoch: 1, Batch: 5, Train Loss: 2.130 Train Accuracy: 66.372%, Validation Loss: 2.060 Validation Accuracy: 67.220%
Validation loss decreased (2.123863 --> 2.059880).  Saving model ...
Epoch: 1, Batch: 6, Train Loss: 2.063 Train Accuracy: 71.018%, Validation Loss: 1.988

In [11]:
def get_model_layerwise_analysis(model):

  for layer, (name, module) in enumerate(model.classifier._modules.items()):
    if name == '0':
      weights = torch.abs(module.weight).cpu()
      print(weights.max(),weights.mean(),weights.std())
    if name == '2':
      weights = torch.abs(module.weight).cpu()
      print(weights.max(),weights.mean(),weights.std())
    if name == '4':
      weights = torch.abs(module.weight).cpu()
      print(weights.max(),weights.mean(),weights.std())

get_model_layerwise_analysis(model)

tensor(0.3333, grad_fn=<MaxBackward1>) tensor(0.0327, grad_fn=<MeanBackward0>) tensor(0.0296, grad_fn=<StdBackward0>)
tensor(0.3553, grad_fn=<MaxBackward1>) tensor(0.0464, grad_fn=<MeanBackward0>) tensor(0.0378, grad_fn=<StdBackward0>)
tensor(0.3024, grad_fn=<MaxBackward1>) tensor(0.0835, grad_fn=<MeanBackward0>) tensor(0.0517, grad_fn=<StdBackward0>)


In [0]:
initial_parameters = sum([p.numel() for p in model.parameters()])

In [0]:
def count_nonzeros(model):
  nonzeros = 0
  for param in model.parameters():
    if param is not None: nonzeros += torch.sum((param != 0).int()).item()
  return(nonzeros)

In [0]:
def update_masks(model, mask_linear1, mask_linear2, mask_linear3, k1, k2, k3):
  
  for layer, (name, module) in enumerate(model.classifier._modules.items()):
    if name == '0':
      a_l1 = 0.25*torch.abs(module.weight.max()).data * k1
      t_l1 = 5*torch.abs(module.weight.std()).data
      b_l1 = a_l1 + t_l1
    if name == '2':
      a_l2 = 0.2*torch.abs(module.weight.max()).data * k2
      t_l2 = 5*torch.abs(module.weight.std()).data
      b_l2 = a_l2 + t_l2
    if name == '4':
      a_l3 = 0.15*torch.abs(module.weight.max()).data * k3
      t_l3 = 5*torch.abs(module.weight.std()).data
      b_l3 = a_l3 + t_l3
    
  for layer, (name, module) in enumerate(model.classifier._modules.items()):
    if name == '0':
      weights = torch.abs(module.weight).cpu()
      temp_mask1 = (weights > b_l1).float() * 1; temp_mask2 = (weights > a_l1).float() * 1
      temp_mask = (temp_mask2 * mask_linear1) + temp_mask1
      mask_linear1 = (temp_mask >= 1.).float() * 1
    if name == '2':
      weights = torch.abs(module.weight).cpu()
      temp_mask1 = (weights > b_l2).float() * 1; temp_mask2 = (weights > a_l2).float() * 1
      temp_mask = (temp_mask2 * mask_linear2) + temp_mask1
      mask_linear2 = (temp_mask >= 1.).float() * 1
    if name == '4':
      weights = torch.abs(module.weight).cpu()
      temp_mask1 = (weights > b_l3).float() * 1; temp_mask2 = (weights > a_l3).float() * 1
      temp_mask = (temp_mask2 * mask_linear3) + temp_mask1
      mask_linear3 = (temp_mask >= 1.).float() * 1

  return mask_linear1, mask_linear2, mask_linear3

In [0]:
def model_surgery(model, mask_linear1, mask_linear2, mask_linear3):
  
  for layer, (name, module) in enumerate(model.classifier._modules.items()):
    if name == '0':
      weights = (module.weight.data) * mask_linear1.to(device)
      model.classifier._modules[name].weight.data = weights.to(device)
    if name == '2':
      weights = (module.weight.data) * mask_linear2.to(device)
      model.classifier._modules[name].weight.data = weights.to(device)
    if name == '4':
      weights = (module.weight.data) * mask_linear3.to(device)
      model.classifier._modules[name].weight.data = weights.to(device)

  for param in model.parameters(): param.requires_grad = True
  
  return(model)

In [0]:
def perform_surgery_training(model,prune_itr,lr_multiplier,mask_linear1,mask_linear2,mask_linear3):
  
  scale_factor = 16

  learning_rate = 1e-3*lr_multiplier; wt_dcy =1e-7; lr_patience = int(500/scale_factor); lr_stepsize = 1; es_patience = int(30000/scale_factor)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=wt_dcy)
  scheduler_plateau = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=lr_patience, verbose=True, threshold=1e-7)
  scheduler_stepLR = StepLR(optimizer, step_size=lr_stepsize, gamma=0.995)
  early_stopping = EarlyStopping(patience=es_patience,filename='checkpoint_'+str(prune_itr), verbose=True)

  torch.cuda.empty_cache()
  model = model.to(device)
  num_epochs = 50
  es_flag = 0

  final_model = deepcopy(model)

  train_loss = []; test_loss = []; train_accuracy = []; test_accuracy = []

  nonzero_parameter_list = []
  
  best_loss = 100000; best_accuracy = 2; best_parameter_size = 0

  prob_itr = 0; prob_threshold = 1.0; k1 = 1; k2 = 1; k3 = 1; k4 = 1 

  for epoch in range(num_epochs):
    
    correct = 0; data_size = 0; t_loss = 0
    
    for i, (images, labels) in enumerate(data_loader_train):

      model.train()
      images = images.to(device); labels = labels.to(device)
      data_size += len(images)
      outputs = torch.nn.functional.log_softmax(model(images), dim=1)
      trainloss = criterion(outputs,labels)
      optimizer.zero_grad()
      trainloss.backward()

      choice = random.random()
      if choice <= prob_threshold:
        print('Performing Surgery for choice as ' + str(round(choice,4)) + ' for threshold ' + str(round(prob_threshold,4)) + '.')
        mask_linear1, mask_linear2, mask_linear3 = update_masks(model, mask_linear1, mask_linear2, mask_linear3, k1, k2, k3)
        k1 = k1 * 1.0011; k2 = k2 * 1.00111; k3 = k3 * 1.00122

      optimizer.step()
      model = model_surgery(model, mask_linear1, mask_linear2, mask_linear3)

      prob_itr += 1
      prob_threshold *= 0.99925
      nonzero_parameters = count_nonzeros(model)
      nonzero_parameter_list.append(nonzero_parameters)
      print('Parameters are: ' + str(nonzero_parameters))
      
      _, predicted = torch.max(outputs.data, 1)
      correct += (predicted == labels).sum().item()
      t_loss += trainloss.item()
    
      correct_t = 0; data_size_t = 0; tt_loss = 0

      model.eval()
      with torch.no_grad():
        for images, labels in data_loader_test:
          images = images.to(device); labels = labels.to(device)
          data_size_t += len(images)
          outputs = torch.nn.functional.log_softmax(model(images), dim=1)
          _, predicted = torch.max(outputs.data, 1)
          correct_t += (predicted == labels).sum().item()
          tt_loss += criterion(outputs, labels).item()
      print('Epoch: {}, Batch: {}, Train Loss: {:.3f} Train Accuracy: {:.3f}%, Validation Loss: {:.3f} Validation Accuracy: {:.3f}%'.
            format(epoch+1, i+1, trainloss.item(), 100*(predicted==labels).sum().item()/len(images), tt_loss/len(data_loader_test), 100*correct_t/data_size_t))
      
      train_loss.append(trainloss.item()); test_loss.append(tt_loss/len(data_loader_test))
      train_accuracy.append((predicted==labels).sum().item()/len(images)); test_accuracy.append(correct_t/data_size_t)
      
      if (tt_loss/len(data_loader_test)) < best_loss:
        best_loss = tt_loss/len(data_loader_test)
        best_accuracy = correct_t/data_size_t
        best_parameter_size = nonzero_parameters
        
      scheduler_stepLR.step()
      scheduler_plateau.step(tt_loss/len(data_loader_test))

      if (correct_t/data_size_t) >= (best_accuracy-(0.1*best_accuracy)):
        last_loss = tt_loss/len(data_loader_test)
        last_accuracy = correct_t/data_size_t
        last_parameter_size = nonzero_parameters
        final_model = deepcopy(model)

      early_stopping(tt_loss/len(data_loader_test), model)
      if early_stopping.early_stop:
        print("Early stopping")
        es_flag = 1
        break
    if es_flag == 1: break

  model.load_state_dict(torch.load(basepath+'/CheckPoints/'+'checkpoint_'+str(prune_itr)+'.pt'))

  return(model, final_model, train_loss, test_loss, train_accuracy, test_accuracy, best_loss, best_accuracy, best_parameter_size, last_loss, last_accuracy, last_parameter_size, nonzero_parameter_list)

In [0]:
def plot_parameters(parameter_list,filename,i):
  x_axis = list(range(1,len(parameter_list)+1))
  fig = plt.figure(i,figsize=(20,10))
  plt.plot(x_axis, parameter_list, color='blue', label='Total Parameters')
  plt.legend(loc=1)
  plt.xlabel('Iterations over entire dataset')
  plt.ylabel('Parameters')
  plt.ylim(0,70000)
  plt.savefig(basepath+'/Parameters/'+filename+'.png')
  plt.close(fig)

In [18]:
lr_multiplier = 1
new_model = deepcopy(model)
#new_model.load_state_dict(torch.load(basepath+'/CheckPoints/'+'checkpoint_1.pt'))
pruned_model, final_model, train_loss, test_loss, train_accuracy, test_accuracy, best_loss, best_accuracy, best_parameter_size, last_loss, last_accuracy, last_parameter_size, parameter_list = perform_surgery_training(new_model,1,lr_multiplier,mask_linear1,mask_linear2,mask_linear3)
dict_best_info[1] = (best_loss,best_accuracy)
plot_losses(train_loss,test_loss,'l1',1)
plot_accuracies(train_accuracy,test_accuracy,'a1',2)
plot_parameters(parameter_list,'p1',3)

pruned_model.load_state_dict(torch.load(basepath+'/CheckPoints/'+'checkpoint_1.pt'))
pruned_model = pruned_model.to(device)
print(pruned_model)

Performing Surgery for choice as 0.0021 for threshold 1.0.
Parameters are: 43532
Epoch: 1, Batch: 1, Train Loss: 0.068 Train Accuracy: 75.774%, Validation Loss: 0.937 Validation Accuracy: 75.330%
Validation loss decreased (inf --> 0.936903).  Saving model ...
Performing Surgery for choice as 0.7735 for threshold 0.9992.
Parameters are: 42928
Epoch: 1, Batch: 2, Train Loss: 0.940 Train Accuracy: 75.608%, Validation Loss: 0.931 Validation Accuracy: 76.180%
Validation loss decreased (0.936903 --> 0.931497).  Saving model ...
Performing Surgery for choice as 0.3646 for threshold 0.9985.
Parameters are: 42520
Epoch: 1, Batch: 3, Train Loss: 0.936 Train Accuracy: 82.080%, Validation Loss: 0.846 Validation Accuracy: 81.380%
Validation loss decreased (0.931497 --> 0.845733).  Saving model ...
Performing Surgery for choice as 0.3179 for threshold 0.9978.
Parameters are: 41994
Epoch: 1, Batch: 4, Train Loss: 0.868 Train Accuracy: 86.504%, Validation Loss: 0.759 Validation Accuracy: 86.880%
Valid

In [19]:
get_model_layerwise_analysis(pruned_model)

tensor(0.3677, grad_fn=<MaxBackward1>) tensor(0.0120, grad_fn=<MeanBackward0>) tensor(0.0364, grad_fn=<StdBackward0>)
tensor(0.3900, grad_fn=<MaxBackward1>) tensor(0.0224, grad_fn=<MeanBackward0>) tensor(0.0541, grad_fn=<StdBackward0>)
tensor(0.3643, grad_fn=<MaxBackward1>) tensor(0.1006, grad_fn=<MeanBackward0>) tensor(0.0791, grad_fn=<StdBackward0>)


In [20]:
remaining_parameters = count_nonzeros(pruned_model)
print('For Compression without loss...')
print('Number of Initial Parameters are ' + str(initial_parameters))
print('Number of Remaining Parameters are ' + str(remaining_parameters))
print('Compression Rate without loss is ' + str(initial_parameters/remaining_parameters))
print('Best Accuracy is ' + str(round(best_accuracy*100,2)))
print('Best Loss is ' + str(round(best_loss,3)))

For Compression without loss...
Number of Initial Parameters are 266610
Number of Remaining Parameters are 30206
Compression Rate without loss is 8.826392107528306
Best Accuracy is 97.09
Best Loss is 0.104


In [21]:
def layer_wise_comparison(model,pruned_model):
  selected_layers = [0,2,4]
  model_layers = []
  for param in model.parameters():
    if param is not None: model_layers.append(torch.sum((param != 0).int()).item())
  
  pruned_model_layers = []
  for param in pruned_model.parameters():
    if param is not None: pruned_model_layers.append(torch.sum((param != 0).int()).item())

  for i in range(len(model_layers)):
    if i in selected_layers:
      print('Layer Parameters: ' + str(model_layers[i]) + ', Parameters Left: ' + str(pruned_model_layers[i]) + ', Percentage Left: ' + str(round(pruned_model_layers[i]*100/model_layers[i],2)))

print('For Compression without loss...')  
layer_wise_comparison(model,pruned_model)

For Compression without loss...
Layer Parameters: 235200, Parameters Left: 24362, Percentage Left: 10.36
Layer Parameters: 30000, Parameters Left: 4722, Percentage Left: 15.74
Layer Parameters: 1000, Parameters Left: 712, Percentage Left: 71.2


In [22]:
print('For Best Compression...')  
layer_wise_comparison(model,final_model)

For Best Compression...
Layer Parameters: 235200, Parameters Left: 3609, Percentage Left: 1.53
Layer Parameters: 30000, Parameters Left: 965, Percentage Left: 3.22
Layer Parameters: 1000, Parameters Left: 602, Percentage Left: 60.2


In [23]:
print('Trained for '+str(last_parameter_size)+' non-zero parameters reaching '+str(initial_parameters/last_parameter_size)+' times compression with loss '+str(round(last_loss,3))+' and accuracy '+str(round(last_accuracy*100,2))+'%.')

Trained for 5586 non-zero parameters reaching 47.72824919441461 times compression with loss 1.21 and accuracy 87.69%.
